In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

# Define the emotions to be considered
valid_emotions = ["Aversion", "Anger", "Peace", "Happiness", "Sadness", "Fear", "Surprise"]

def modify_image_folder_path(image_folder, valid_emotion):
    # Replace "ThirdPart" with the valid emotion
    modified_image_folder = image_folder.replace("ThirdPart", valid_emotion)
    return modified_image_folder

def process_and_crop_images(csv_data):
    for index, row in csv_data.iterrows():
        image_folder = row['Video Path']
        emotion_values = row['Emotion Value'].split(',')
        start_frame = row['Start Frame']
        end_frame = row['End Frame']
        npy_path = row['Npy Path']
        
        # Process each emotion value separately
        for emotion in emotion_values:
            valid_emotion = emotion.strip()
            if valid_emotion in valid_emotions:
                # Modify the image folder path based on the valid emotion
                modified_image_folder = modify_image_folder_path(image_folder, valid_emotion)
                
                # Check if the modified image folder exists
                if not os.path.exists(modified_image_folder):
                    print(f"Error: Modified image folder '{modified_image_folder}' does not exist for emotion '{valid_emotion}'.")
                    continue
                
                # Debugging: Print the modified image folder path
                print(f"Processing images for emotion '{valid_emotion}' in folder '{modified_image_folder}'")

                # Load the .npy file for this specific folder
                if not os.path.exists(npy_path):
                    print(f"Error: NPY file '{npy_path}' does not exist for emotion '{valid_emotion}'.")
                    continue
                
                npy_data = np.load(npy_path)
                
                # Calculate the number of frames to process
                frame_diff = end_frame - start_frame
                
                # Filter the npy data for the given frames
                frame_data = npy_data[start_frame:end_frame+1]
                
                for frame in frame_data:
                    frame_number = int(frame[0])
                    person_id = int(frame[1])
                    joint_coordinates = frame[2:].reshape((18, 3))
                    
                    # Load the corresponding image
                    image_path = os.path.join(modified_image_folder, f"{frame_number:06d}.jpg")
                    
                    # Debugging: Print the image path being checked
                    print(f"Looking for image at path '{image_path}'")
                    
                    if not os.path.exists(image_path):
                        print(f"Error: Image '{image_path}' does not exist for frame '{frame_number}' and emotion '{valid_emotion}'.")
                        continue
                    
                    try:
                        image = Image.open(image_path)
                    except Exception as e:
                        print(f"Error: Failed to open image '{image_path}' for frame '{frame_number}' and emotion '{valid_emotion}'. Exception: {e}")
                        continue
                    
                    cropped_images = crop_image(image, joint_coordinates)
                    
                    if not cropped_images:
                        print(f"Error: No cropped images generated for frame '{frame_number}' and emotion '{valid_emotion}'.")
                    
                    # Save the cropped images
                    for i, cropped_image in enumerate(cropped_images):
                        cropped_image_path = os.path.join(modified_image_folder, f"{frame_number:06d}_person_{person_id}_joint_{i}.jpg")
                        try:
                            cropped_image.save(cropped_image_path)
                        except Exception as e:
                            print(f"Error: Failed to save cropped image '{cropped_image_path}' for frame '{frame_number}', person '{person_id}', joint '{i}' and emotion '{valid_emotion}'. Exception: {e}")
            else:
                # Print an error message if no valid emotion is found
                print(f"Error: '{valid_emotion}' is not a valid emotion for row {index}. Skipping this emotion.")

def crop_image(image, joint_coordinates):
    cropped_images = []
    for i, joint in enumerate(joint_coordinates):
        x, y, confidence = joint
        if confidence > 0.5:  # Assuming a threshold for confidence
            left = max(0, x - 50)
            top = max(0, y - 50)
            right = min(image.width, x + 50)
            bottom = min(image.height, y + 50)
            cropped_image = image.crop((left, top, right, bottom))
            cropped_images.append(cropped_image)
    return cropped_images

# Load the CSV file
csv_file_path = r"E:\dummy.csv"
csv_data = pd.read_csv(csv_file_path)

# Process and crop images
process_and_crop_images(csv_data)


In [14]:
import numpy as np
import pandas as pd

# Load the .npy file
data = np.load(r"D:\BoLD\BOLD_public\joints\003\IzvOYVMltkI.mp4\0114.npy")

# Convert the numpy array to a pandas DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a .csv file
df.to_csv('data3.csv', index=False)


In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

# Define the emotions to be considered
valid_emotions = ["Aversion", "Anger", "Peace", "Happiness", "Sadness", "Fear", "Surprise"]

def modify_image_folder_path(image_folder, valid_emotion):
    # Replace "ThirdPart" with the valid emotion
    modified_image_folder = image_folder.replace("ThirdPart", valid_emotion)
    return modified_image_folder

def process_and_crop_images(csv_data):
    for index, row in csv_data.iterrows():
        image_folder = row['Video Path']
        emotion_values = row['Emotion Value'].split(',')
        start_frame = row['Start Frame']
        end_frame = row['End Frame']
        npy_path = row['Npy Path']
        
        # Process each emotion value separately
        for emotion in emotion_values:
            valid_emotion = emotion.strip()
            if valid_emotion in valid_emotions:
                # Modify the image folder path based on the valid emotion
                modified_image_folder = modify_image_folder_path(image_folder, valid_emotion)
                
                # Check if the modified image folder exists
                if not os.path.exists(modified_image_folder):
                    print(f"Error: Modified image folder '{modified_image_folder}' does not exist for emotion '{valid_emotion}'.")
                    continue
                
                # Debugging: Print the modified image folder path
                print(f"Processing images for emotion '{valid_emotion}' in folder '{modified_image_folder}'")

                # Load the .npy file for this specific folder
                if not os.path.exists(npy_path):
                    print(f"Error: NPY file '{npy_path}' does not exist for emotion '{valid_emotion}'.")
                    continue
                
                npy_data = np.load(npy_path)
                
                # Calculate the number of frames to process
                frame_count = end_frame - start_frame + 1  # Include both start and end frames
                
                # Ensure we don't exceed the available data in npy file
                if frame_count > len(npy_data):
                    print(f"Error: The number of frames to process ({frame_count}) exceeds the available data in the npy file ({len(npy_data)}).")
                    continue
                
                # Process the frames within the specified range
                for i in range(frame_count):
                    frame_number = start_frame + i
                    frame = npy_data[i]  # Get the i-th row from the npy file
                    person_id = int(frame[1])
                    joint_coordinates = frame[2:].reshape((18, 3))
                    
                    # Load the corresponding image
                    image_path = os.path.join(modified_image_folder, f"frame_{frame_number}.jpg")
                    
                    # Debugging: Print the image path being checked
                    print(f"Looking for image at path '{image_path}'")
                    
                    if not os.path.exists(image_path):
                        print(f"Error: Image '{image_path}' does not exist for frame '{frame_number}' and emotion '{valid_emotion}'.")
                        continue
                    
                    try:
                        image = Image.open(image_path)
                    except Exception as e:
                        print(f"Error: Failed to open image '{image_path}' for frame '{frame_number}' and emotion '{valid_emotion}'. Exception: {e}")
                        continue
                    
                    cropped_image = crop_image(image, joint_coordinates)
                    
                    if cropped_image is None:
                        print(f"Error: No cropped images generated for frame '{frame_number}' and emotion '{valid_emotion}'.")
                    
                    # Save the cropped image
                    cropped_image_path = image_path  # Save the cropped image to the original path
                    try:
                        cropped_image.save(cropped_image_path)
                    except Exception as e:
                        print(f"Error: Failed to save cropped image '{cropped_image_path}' for frame '{frame_number}', person '{person_id}' and emotion '{valid_emotion}'. Exception: {e}")
            else:
                # Print an error message if no valid emotion is found
                print(f"Error: '{valid_emotion}' is not a valid emotion for row {index}. Skipping this emotion.")

def crop_image(image, joint_coordinates):
    # Determine the bounding box for all joints
    min_x = min(joint_coordinates[:, 0])
    min_y = min(joint_coordinates[:, 1])
    max_x = max(joint_coordinates[:, 0])
    max_y = max(joint_coordinates[:, 1])

    # Ensure coordinates are within image bounds
    top_left_x = max(int(min_x), 0)
    top_left_y = max(int(min_y), 0)
    bottom_right_x = min(int(max_x), image.width)
    bottom_right_y = min(int(max_y), image.height)

    # Crop image
    cropped_image = image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))
    return cropped_image

# Load the CSV file
csv_file_path = r"E:\annotation_train_2.0.csv"
csv_data = pd.read_csv(csv_file_path)

# Process and crop images
process_and_crop_images(csv_data)


In [4]:
import cv2
import numpy as np
import os

# Load YOLO
net = cv2.dnn.readNet(r"D:\My Files\Downloads - V\yolov3.weights", r"D:\My Files\Downloads - V\yolov3.cfg")
layer_names = net.getLayerNames()

# Adjust for different return types of getUnconnectedOutLayers
try:
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
except IndexError:
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO labels
with open(r"D:\My Files\Downloads - V\coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

def detect_and_crop(image_path, output_path):
    # Read the image
    img = cv2.imread(image_path)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Information to show on the screen
    class_ids = []
    confidences = []
    boxes = []

    # For each detection from each output layer, get the confidence, class id, bounding box params
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "person":  # Filter for 'person'
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-max suppression to eliminate redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Loop through the detected boxes
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            # Crop the detected human
            cropped_img = img[y:y+h, x:x+w]
            # Save the cropped image
            output_filename = os.path.join(output_path, os.path.basename(image_path))
            cv2.imwrite(output_filename, cropped_img)

# Example usage
input_folder = r"E:\cropped_images\video_110"
output_folder = r"E:\crop"

# Process all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        input_image_path = os.path.join(input_folder, filename)
        detect_and_crop(input_image_path, output_folder)


# Crop images if it recognizes as human


In [ ]:
# YOLO Detection with Cropping based on Joints coordinates


import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2

# Define the emotions to be considered
valid_emotions = ["Aversion", "Anger", "Peace", "Happiness", "Sadness", "Fear", "Surprise"]

def modify_image_folder_path(image_folder, valid_emotion):
    # Replace "ThirdPart" with the valid emotion
    modified_image_folder = image_folder.replace("ThirdPart", valid_emotion)
    return modified_image_folder

def process_and_crop_images(csv_data):
    for index, row in csv_data.iterrows():
        image_folder = row['Video Path']
        emotion_values = row['Emotion Value'].split(',')
        start_frame = row['Start Frame']
        end_frame = row['End Frame']
        npy_path = row['Npy Path']
        
        # Process each emotion value separately
        for emotion in emotion_values:
            valid_emotion = emotion.strip()
            if valid_emotion in valid_emotions:
                # Modify the image folder path based on the valid emotion
                modified_image_folder = modify_image_folder_path(image_folder, valid_emotion)
                
                # Check if the modified image folder exists
                if not os.path.exists(modified_image_folder):
                    print(f"Error: Modified image folder '{modified_image_folder}' does not exist for emotion '{valid_emotion}'.")
                    continue
                
                # Debugging: Print the modified image folder path
                print(f"Processing images for emotion '{valid_emotion}' in folder '{modified_image_folder}'")

                # Load the .npy file for this specific folder
                if not os.path.exists(npy_path):
                    print(f"Error: NPY file '{npy_path}' does not exist for emotion '{valid_emotion}'.")
                    continue
                
                npy_data = np.load(npy_path)
                
                # Calculate the number of frames to process
                frame_count = end_frame - start_frame + 1  # Include both start and end frames
                
                # Ensure we don't exceed the available data in npy file
                if frame_count > len(npy_data):
                    print(f"Error: The number of frames to process ({frame_count}) exceeds the available data in the npy file ({len(npy_data)}).")
                    continue
                
                # Process the frames within the specified range
                for i in range(frame_count):
                    frame_number = start_frame + i
                    frame = npy_data[i]  # Get the i-th row from the npy file
                    person_id = int(frame[1])
                    joint_coordinates = frame[2:].reshape((18, 3))
                    
                    # Load the corresponding image
                    image_path = os.path.join(modified_image_folder, f"frame_{frame_number}.jpg")
                    
                    # Debugging: Print the image path being checked
                    print(f"Looking for image at path '{image_path}'")
                    
                    if not os.path.exists(image_path):
                        print(f"Error: Image '{image_path}' does not exist for frame '{frame_number}' and emotion '{valid_emotion}'.")
                        continue
                    
                    try:
                        image = Image.open(image_path)
                    except Exception as e:
                        print(f"Error: Failed to open image '{image_path}' for frame '{frame_number}' and emotion '{valid_emotion}'. Exception: {e}")
                        continue
                    
                    # Detect and crop the human closest to the joint coordinates
                    cropped_image = detect_and_crop(image, joint_coordinates)
                    
                    if cropped_image is None:
                        print(f"Error: No cropped images generated for frame '{frame_number}' and emotion '{valid_emotion}'.")
                        continue
                    
                    # Save the cropped image
                    cropped_image_path = image_path  # Save the cropped image to the original path
                    try:
                        cropped_image.save(cropped_image_path)
                    except Exception as e:
                        print(f"Error: Failed to save cropped image '{cropped_image_path}' for frame '{frame_number}', person '{person_id}' and emotion '{valid_emotion}'. Exception: {e}")
            else:
                # Print an error message if no valid emotion is found
                print(f"Error: '{valid_emotion}' is not a valid emotion for row {index}. Skipping this emotion.")

def crop_image(image, joint_coordinates):
    # Determine the bounding box for all joints
    min_x = min(joint_coordinates[:, 0])
    min_y = min(joint_coordinates[:, 1])
    max_x = max(joint_coordinates[:, 0])
    max_y = max(joint_coordinates[:, 1])

    # Ensure coordinates are within image bounds
    top_left_x = max(int(min_x), 0)
    top_left_y = max(int(min_y), 0)
    bottom_right_x = min(int(max_x), image.width)
    bottom_right_y = min(int(max_y), image.height)

    # Crop image
    cropped_image = image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))
    return cropped_image

# Load YOLO
net = cv2.dnn.readNet(r"E:\YOLO\yolov3.weights", r"E:\YOLO\yolov3.cfg")
layer_names = net.getLayerNames()

# Adjust for different return types of getUnconnectedOutLayers
try:
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
except IndexError:
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO labels
with open(r"E:\YOLO\coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

def detect_and_crop(image, joint_coordinates):
    # Convert PIL image to OpenCV format
    img = np.array(image)
    img = img[:, :, ::-1].copy()  # RGB to BGR for OpenCV

    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Information to show on the screen
    class_ids = []
    confidences = []
    boxes = []

    # For each detection from each output layer, get the confidence, class id, bounding box params
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "person":  # Filter for 'person'
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-max suppression to eliminate redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    if len(indexes) == 0:
        return None

    # Find the closest bounding box to the joint coordinates
    min_distance = float('inf')
    best_box = None

    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        box_center_x = x + w / 2
        box_center_y = y + h / 2

        # Calculate the distance between the box center and the joint coordinates
        distance = np.sqrt((box_center_x - np.mean(joint_coordinates[:, 0]))**2 +
                           (box_center_y - np.mean(joint_coordinates[:, 1]))**2)

        if distance < min_distance:
            min_distance = distance
            best_box = (x, y, w, h)

    if best_box is not None:
        x, y, w, h = best_box
        cropped_img = img[y:y+h, x:x+w]

        # Convert back to PIL image
        cropped_img = Image.fromarray(cropped_img[:, :, ::-1])  # BGR to RGB for PIL
        return cropped_img
    else:
        return None

# Load the CSV file
csv_file_path = r"E:\dummy.csv"
csv_data = pd.read_csv(csv_file_path)

# Process and crop images
process_and_crop_images(csv_data)


In [ ]:
# YOLO Detection with Cropping based on Joints coordinates
# Error fixed

import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2

# Define the emotions to be considered
valid_emotions = ["Aversion", "Anger", "Peace", "Happiness", "Sadness", "Fear", "Surprise"]

def modify_image_folder_path(image_folder, valid_emotion):
    # Replace "ThirdPart" with the valid emotion
    modified_image_folder = image_folder.replace("ThirdPart", valid_emotion)
    return modified_image_folder

def process_and_crop_images(csv_data):
    for index, row in csv_data.iterrows():
        image_folder = row['Video Path']
        emotion_values = row['Emotion Value'].split(',')
        start_frame = row['Start Frame']
        end_frame = row['End Frame']
        npy_path = row['Npy Path']
        
        # Process each emotion value separately
        for emotion in emotion_values:
            valid_emotion = emotion.strip()
            if valid_emotion in valid_emotions:
                # Modify the image folder path based on the valid emotion
                modified_image_folder = modify_image_folder_path(image_folder, valid_emotion)
                
                # Check if the modified image folder exists
                if not os.path.exists(modified_image_folder):
                    print(f"Error: Modified image folder '{modified_image_folder}' does not exist for emotion '{valid_emotion}'.")
                    continue
                
                # Debugging: Print the modified image folder path
                print(f"Processing images for emotion '{valid_emotion}' in folder '{modified_image_folder}'")

                # Load the .npy file for this specific folder
                if not os.path.exists(npy_path):
                    print(f"Error: NPY file '{npy_path}' does not exist for emotion '{valid_emotion}'.")
                    continue
                
                npy_data = np.load(npy_path)
                
                # Calculate the number of frames to process
                frame_count = end_frame - start_frame + 1  # Include both start and end frames
                
                # Ensure we don't exceed the available data in npy file
                if frame_count > len(npy_data):
                    print(f"Error: The number of frames to process ({frame_count}) exceeds the available data in the npy file ({len(npy_data)}).")
                    continue
                
                # Process the frames within the specified range
                for i in range(frame_count):
                    frame_number = start_frame + i
                    frame = npy_data[i]  # Get the i-th row from the npy file
                    person_id = int(frame[1])
                    joint_coordinates = frame[2:].reshape((18, 3))
                    
                    # Load the corresponding image
                    image_path = os.path.join(modified_image_folder, f"frame_{frame_number}.jpg")
                    
                    # Debugging: Print the image path being checked
                    print(f"Looking for image at path '{image_path}'")
                    
                    if not os.path.exists(image_path):
                        print(f"Error: Image '{image_path}' does not exist for frame '{frame_number}' and emotion '{valid_emotion}'.")
                        continue
                    
                    try:
                        image = Image.open(image_path)
                    except Exception as e:
                        print(f"Error: Failed to open image '{image_path}' for frame '{frame_number}' and emotion '{valid_emotion}'. Exception: {e}")
                        continue
                    
                    # Detect and crop the human closest to the joint coordinates
                    cropped_image = detect_and_crop(image, joint_coordinates)
                    
                    if cropped_image is None:
                        print(f"Error: No cropped images generated for frame '{frame_number}' and emotion '{valid_emotion}'.")
                        continue
                    
                    # Save the cropped image
                    cropped_image_path = image_path  # Save the cropped image to the original path
                    try:
                        cropped_image.save(cropped_image_path)
                    except Exception as e:
                        print(f"Error: Failed to save cropped image '{cropped_image_path}' for frame '{frame_number}', person '{person_id}' and emotion '{valid_emotion}'. Exception: {e}")
            else:
                # Print an error message if no valid emotion is found
                print(f"Error: '{valid_emotion}' is not a valid emotion for row {index}. Skipping this emotion.")

# Updated detect_and_crop function with boundary checking
def detect_and_crop(image, joint_coordinates):
    # Convert PIL image to OpenCV format
    img = np.array(image)
    img = img[:, :, ::-1].copy()  # RGB to BGR for OpenCV

    height, width, _ = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Information to show on the screen
    class_ids = []
    confidences = []
    boxes = []

    # For each detection from each output layer, get the confidence, class id, bounding box params
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "person":  # Filter for 'person'
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-max suppression to eliminate redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    if len(indexes) == 0:
        return None

    # Find the closest bounding box to the joint coordinates
    min_distance = float('inf')
    best_box = None

    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        box_center_x = x + w / 2
        box_center_y = y + h / 2

        # Calculate the distance between the box center and the joint coordinates
        distance = np.sqrt((box_center_x - np.mean(joint_coordinates[:, 0]))**2 +
                           (box_center_y - np.mean(joint_coordinates[:, 1]))**2)

        if distance < min_distance:
            min_distance = distance
            best_box = (x, y, w, h)

    if best_box is not None:
        x, y, w, h = best_box

        # Clamp the coordinates to be within the image boundaries
        x = max(0, x)
        y = max(0, y)
        w = min(w, width - x)
        h = min(h, height - y)

        cropped_img = img[y:y+h, x:x+w]

        # Convert back to PIL image
        cropped_img = Image.fromarray(cropped_img[:, :, ::-1])  # BGR to RGB for PIL
        return cropped_img
    else:
        return None

# Load YOLO
net = cv2.dnn.readNet(r"E:\YOLO\yolov3.weights", r"E:\YOLO\yolov3.cfg")
layer_names = net.getLayerNames()

# Adjust for different return types of getUnconnectedOutLayers
try:
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
except IndexError:
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO labels
with open(r"E:\YOLO\coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load the CSV file
csv_file_path = r"E:\annotation_train_2.0.csv"
csv_data = pd.read_csv(csv_file_path)

# Process and crop images
process_and_crop_images(csv_data)


In [1]:
import os
import shutil

# Path to the main folder containing emotion folders
main_folder_path = r"E:\train_gen_joints"

# List of emotion folders
emotions = ['Anger', 'Happiness', 'Sadness', 'Aversion', 'Peace', 'Surprise', 'Fear']

for emotion in emotions:
    emotion_folder_path = os.path.join(main_folder_path, emotion)
    
    for root, dirs, files in os.walk(emotion_folder_path, topdown=False):
        for file in files:
            file_path = os.path.join(root, file)
            new_path = os.path.join(emotion_folder_path, file)
            
            # If the file already exists, add a unique suffix
            if os.path.exists(new_path):
                base, ext = os.path.splitext(file)
                counter = 1
                new_name = f"{base}_{counter}{ext}"
                new_path = os.path.join(emotion_folder_path, new_name)
                while os.path.exists(new_path):
                    counter += 1
                    new_name = f"{base}_{counter}{ext}"
                    new_path = os.path.join(emotion_folder_path, new_name)
            
            # Move the file to the emotion folder
            shutil.move(file_path, new_path)
        
        # Remove empty directories
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            if not os.listdir(dir_path):
                os.rmdir(dir_path)
